# Load Data từ Bronze Layer sang Silver Layer

Notebook này sẽ đọc dữ liệu từ Bronze layer (MinIO) và xử lý để load vào các bảng Iceberg trong Silver layer với Nessie catalog.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Load_Bronze_To_Silver")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")
print("Spark Session đã được khởi tạo với Nessie catalog!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/05 17:51:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/05 17:51:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Session đã được khởi tạo với Nessie catalog!


## 2. Load Bảng SCHOOL

In [2]:
print("=" * 80)
print("LOAD BẢNG SCHOOL")
print("=" * 80)

# Đọc và merge tất cả các năm
years = [2021, 2022, 2023, 2024, 2025]
base_path = "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_"
df_school = spark.read.option("header", "true").option("inferSchema", "true").csv([f"{base_path}{year}.csv" for year in years]).select("TenTruong", "MaTruong", "TinhThanh").dropDuplicates()

# Transform
df_school_silver = df_school.select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("TenTruong").cast("string").alias("schoolName"),
    col("TinhThanh").cast("string").alias("province"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("schoolId").isNotNull() & col("schoolName").isNotNull())

# Ghi vào Silver
df_school_silver.writeTo("nessie.silver_tables.school").using("iceberg").createOrReplace()
print(f"Đã ghi {df_school_silver.count()} dòng vào school")

# Verify
spark.table("nessie.silver_tables.school").show(5, truncate=False)

LOAD BẢNG SCHOOL


25/11/05 17:51:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/11/05 17:52:03 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to silver_tables/school_da6c8741-f795-4e51-a080-b8e49de5b512/data/00000-11-24a58483-9e44-4432-a570-4896dc636f8b-0-00001.parquet. This is unsupported
                                                                                

Đã ghi 270 dòng vào school
+--------+--------------------------------------+-----------+-------------------------+-------------------------+
|schoolId|schoolName                            |province   |created_at               |updated_at               |
+--------+--------------------------------------+-----------+-------------------------+-------------------------+
|DHF     |Đại học Ngoại Ngữ - Đại học Huế       |Huế        |2025-11-05 17:51:59.36026|2025-11-05 17:51:59.36026|
|DVB     |Đại học Việt Bắc                      |Thái Nguyên|2025-11-05 17:51:59.36026|2025-11-05 17:51:59.36026|
|DCQ     |Đại học Công Nghệ và Quản Lý Hữu Nghị |Hà Nội     |2025-11-05 17:51:59.36026|2025-11-05 17:51:59.36026|
|NTT     |Đại học Nguyễn Tất Thành              |TP HCM     |2025-11-05 17:51:59.36026|2025-11-05 17:51:59.36026|
|KGH     |Trường Sĩ Quan Không Quân - Hệ Đại học|Khánh Hòa  |2025-11-05 17:51:59.36026|2025-11-05 17:51:59.36026|
+--------+--------------------------------------+-----------+

## 3. Load Bảng MAJOR

In [3]:
print("=" * 80)
print("LOAD BẢNG MAJOR")
print("=" * 80)

# --- MAJOR ---
df_major = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/danh sách các ngành đại học/Danh_sách_các_ngành.csv")
df_major_silver = df_major.select(
    col(df_major.columns[0]).cast("string").alias("majorId"),
    col(df_major.columns[1]).cast("string").alias("majorName"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("majorId").isNotNull() & col("majorName").isNotNull()).dropDuplicates(["majorId"])
df_major_silver.writeTo("nessie.silver_tables.major").using("iceberg").createOrReplace()
print(f"Đã ghi {df_major_silver.count()} dòng vào major")

# Verify
spark.table("nessie.silver_tables.major").show(5, truncate=False)

LOAD BẢNG MAJOR


Đã ghi 3265 dòng vào major
+-------+------------------------------------------------------------+--------------------------+--------------------------+
|majorId|majorName                                                   |created_at                |updated_at                |
+-------+------------------------------------------------------------+--------------------------+--------------------------+
|106    |Khoa học Máy tính                                           |2025-11-05 17:52:07.352457|2025-11-05 17:52:07.352457|
|107    |Kỹ thuật Máy tính                                           |2025-11-05 17:52:07.352457|2025-11-05 17:52:07.352457|
|108    |Điện - Điện tử - Viễn Thông - Tự động hoá - Thiết kế vi mạch|2025-11-05 17:52:07.352457|2025-11-05 17:52:07.352457|
|109    |Kỹ Thuật Cơ khí                                             |2025-11-05 17:52:07.352457|2025-11-05 17:52:07.352457|
|110    |Kỹ Thuật Cơ Điện tử                                         |2025-11-05 17:52:07.352457|2

## 4. Load Bảng SUBJECT_GROUP và SUBJECT

In [4]:
print("=" * 80)
print("LOAD BẢNG SUBJECT_GROUP và SUBJECT")
print("=" * 80)

# Đọc file tohop_mon_fixed.csv
df_tohop = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/tohop_mon.csv")

# --- SUBJECT_GROUP ---
df_subject_group_silver = df_tohop.select(
    col(df_tohop.columns[0]).cast("int").alias("subjectGroupId"),
    col(df_tohop.columns[1]).cast("string").alias("subjectGroupName"),
    col(df_tohop.columns[2]).cast("string").alias("subjectCombination"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("subjectGroupId").isNotNull() & col("subjectGroupName").isNotNull() & col("subjectCombination").isNotNull()).dropDuplicates(["subjectGroupName", "subjectCombination"])
df_subject_group_silver.writeTo("nessie.silver_tables.subject_group").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_group_silver.count()} dòng vào subject_group")

# --- SUBJECT ---
df_subject = df_tohop.select(explode(split(col(df_tohop.columns[2]), "-")).alias("subjectName")).withColumn("subjectName", trim(col("subjectName"))).filter(col("subjectName").isNotNull() & (col("subjectName") != "")).distinct()
window_spec = Window.orderBy("subjectName")
df_subject_silver = df_subject.withColumn("subjectId", row_number().over(window_spec)).select(
    col("subjectId").cast("int"),
    col("subjectName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_subject_silver.writeTo("nessie.silver_tables.subject").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_silver.count()} dòng vào subject")

# Verify
spark.table("nessie.silver_tables.subject_group").show(5, truncate=False)
spark.table("nessie.silver_tables.subject").show(5, truncate=False)

LOAD BẢNG SUBJECT_GROUP và SUBJECT
Đã ghi 211 dòng vào subject_group


25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 1

Đã ghi 17 dòng vào subject
+--------------+----------------+------------------+-------------------------+-------------------------+
|subjectGroupId|subjectGroupName|subjectCombination|created_at               |updated_at               |
+--------------+----------------+------------------+-------------------------+-------------------------+
|11            |C04             |Văn-Toán-Địa      |2025-11-05 17:52:12.38055|2025-11-05 17:52:12.38055|
|210           |DH5             |Văn-Sử-Ngoại ngữ  |2025-11-05 17:52:12.38055|2025-11-05 17:52:12.38055|
|40            |C03             |Văn-Toán-Sử       |2025-11-05 17:52:12.38055|2025-11-05 17:52:12.38055|
|71            |C05             |Văn-Lí-Hóa        |2025-11-05 17:52:12.38055|2025-11-05 17:52:12.38055|
|92            |D42             |Văn-Địa-Ngoại ngữ |2025-11-05 17:52:12.38055|2025-11-05 17:52:12.38055|
+--------------+----------------+------------------+-------------------------+-------------------------+
only showing top 5 rows

+--

## 5. Load Bảng SELECTION_METHOD

In [5]:
print("=" * 80)
print("LOAD BẢNG SELECTION_METHOD")
print("=" * 80)

# Đọc từ file benchmark để lấy các phương thức xét tuyển
df_benchmark = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")

# Lấy PhuongThuc và loại bỏ "năm ..."
df_selection = df_benchmark.select(trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")).alias("selectionMethodName")).filter(col("selectionMethodName").isNotNull() & (col("selectionMethodName") != "")).distinct()

window_spec = Window.orderBy("selectionMethodName")
df_selection_method_silver = df_selection.withColumn("selectionMethodId", row_number().over(window_spec)).select(
    col("selectionMethodId").cast("int"),
    col("selectionMethodName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_selection_method_silver.writeTo("nessie.silver_tables.selection_method").using("iceberg").createOrReplace()
print(f"Đã ghi {df_selection_method_silver.count()} dòng vào selection_method")

# Verify
spark.table("nessie.silver_tables.selection_method").show(10, truncate=False)

LOAD BẢNG SELECTION_METHOD


25/11/05 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 17:52:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/05 1

Đã ghi 13 dòng vào selection_method
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|selectionMethodId|selectionMethodName                                   |created_at                |updated_at                |
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|1                |Điểm chuẩn theo phương thức Chứng chỉ quốc tế         |2025-11-05 17:52:18.699449|2025-11-05 17:52:18.699449|
|2                |Điểm chuẩn theo phương thức Điểm học bạ               |2025-11-05 17:52:18.699449|2025-11-05 17:52:18.699449|
|3                |Điểm chuẩn theo phương thức Điểm thi THPT             |2025-11-05 17:52:18.699449|2025-11-05 17:52:18.699449|
|4                |Điểm chuẩn theo phương thức Điểm thi riêng            |2025-11-05 17:52:18.699449|2025-11-05 17:52:18.699449|
|5                |Điểm chuẩn theo phương thức Điểm xét tuyển

## 6. Load Bảng BENCHMARK

In [6]:
print("=" * 80)
print("LOAD BẢNG BENCHMARK")
print("=" * 80)

# Đọc dữ liệu
df_benchmark = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")

# Xử lý PhuongThuc
df_benchmark = df_benchmark.withColumn("PhuongThuc_cleaned", trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")))

# Join với selection_method và subject_group
df_selection_lookup = spark.table("nessie.silver_tables.selection_method")
df_subject_group_lookup = spark.table("nessie.silver_tables.subject_group")

df_benchmark_silver = df_benchmark.join(df_selection_lookup, df_benchmark["PhuongThuc_cleaned"] == df_selection_lookup["selectionMethodName"], "left").join(df_subject_group_lookup, df_benchmark["KhoiThi"] == df_subject_group_lookup["subjectGroupName"], "left").select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("MaNganh").cast("string").alias("majorId"),
    col("subjectGroupId").cast("int"),
    col("selectionMethodId").cast("int"),
    col("Nam").cast("int").alias("year"),
    col("DiemChuan").cast("double").alias("score"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("schoolId").isNotNull() & col("majorId").isNotNull() & col("year").isNotNull() & col("score").isNotNull() & col("selectionMethodId").isNotNull() & col("subjectGroupId").isNotNull()).withColumn("benchmarkId", monotonically_increasing_id().cast("int")).select("benchmarkId", "schoolId", "majorId", "subjectGroupId", "selectionMethodId", "year", "score", "created_at", "updated_at").dropDuplicates(["schoolId", "majorId", "subjectGroupId", "selectionMethodId", "year"])

df_benchmark_silver.writeTo("nessie.silver_tables.benchmark").using("iceberg").createOrReplace()
print(f"Đã ghi {df_benchmark_silver.count()} dòng vào benchmark")

# Verify
spark.table("nessie.silver_tables.benchmark").show(5, truncate=False)
spark.table("nessie.silver_tables.benchmark").groupBy("year").count().orderBy("year").show()

LOAD BẢNG BENCHMARK


Đã ghi 147525 dòng vào benchmark
+-----------+--------+---------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|benchmarkId|schoolId|majorId  |subjectGroupId|selectionMethodId|year|score|created_at                |updated_at                |
+-----------+--------+---------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|207        |BKA     |TE3      |3             |3                |2021|26.48|2025-11-05 17:52:24.263663|2025-11-05 17:52:24.263663|
|253        |BVH     |7329001  |1             |3                |2021|26.35|2025-11-05 17:52:24.263663|2025-11-05 17:52:24.263663|
|301        |BVU     |7220201  |8             |2                |2021|18.0 |2025-11-05 17:52:24.263663|2025-11-05 17:52:24.263663|
|632        |DAD     |7140202  |1             |3                |2021|19.0 |2025-11-05 17:52:24.263663|2025-11-05 17:52:24.263663|
|869        |DBH     |7520207.0|1             |3  

## 7. Load Bảng REGION

In [7]:
print("=" * 80)
print("LOAD BẢNG REGION")
print("=" * 80)

df_region = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/region.csv")
df_region_silver = df_region.select(
    lpad(col(df_region.columns[0]).cast("string"), 2, "0").alias("regionId"),  # Format thành 2 chữ số: "1" -> "01"
    col(df_region.columns[1]).cast("string").alias("regionName"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("regionId").isNotNull() & col("regionName").isNotNull()).dropDuplicates(["regionId"])

df_region_silver.writeTo("nessie.silver_tables.region").using("iceberg").createOrReplace()
print(f"Đã ghi {df_region_silver.count()} dòng vào region")

# Verify
spark.table("nessie.silver_tables.region").show(10, truncate=False)

LOAD BẢNG REGION
Đã ghi 64 dòng vào region
+--------+-----------------------+-------------------------+-------------------------+
|regionId|regionName             |created_at               |updated_at               |
+--------+-----------------------+-------------------------+-------------------------+
|01      |Sở GDĐT Hà Nội         |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|02      |Sở GDĐT TP. Hồ Chí Minh|2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|03      |Sở GDĐT Hải Phòng      |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|04      |Sở GDĐT Đà Nẵng        |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|05      |Sở GDĐT Hà Giang       |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|06      |Sở GDĐT Cao Bằng       |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|07      |Sở GDĐT Lai Châu       |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|
|08      |Sở GDĐT Lào Cai        |2025-11-05 17:52:33.96881|2025-11-05 17:52:33.96881|


## 8. Load Bảng STUDENT_SCORES

In [8]:
print("=" * 80)
print("LOAD BẢNG STUDENT_SCORES")
print("=" * 80)

# Đọc từ nhiều năm
years = [2021, 2022, 2023, 2024, 2025]
all_dfs = []
for year in years:
    try:
        df_year = spark.read.option("header", "true").option("inferSchema", "false").option("encoding", "UTF-8").csv(f"s3a://bronze/structured_data/điểm từng thí sinh/{year}/*.csv").withColumn("Year", lit(year))
        all_dfs.append(df_year)
        print(f"Đọc được {df_year.count():,} dòng từ năm {year}")
    except:
        print(f"Không tìm thấy dữ liệu năm {year}")

df_scores = all_dfs[0]
for df in all_dfs[1:]:
    df_scores = df_scores.union(df)

# Đọc bảng subject để map tên môn -> subjectId
df_subject_lookup = spark.table("nessie.silver_tables.subject").select("subjectId", "subjectName")
subject_map = {row.subjectName: row.subjectId for row in df_subject_lookup.collect()}
print(f"\nĐã load {len(subject_map)} môn học để mapping")

# UDF để parse điểm và map với subjectId
from typing import Dict
def parse_scores_with_subject_id(score_string: str) -> Dict[int, float]:
    if not score_string or score_string.strip() == "":
        return {}
    scores_dict = {}
    try:
        pairs = score_string.split(",")
        for pair in pairs:
            if ":" in pair:
                subject_name, score = pair.split(":")
                subject_name = subject_name.strip()
                # Map tên môn -> subjectId
                if subject_name in subject_map:
                    subject_id = subject_map[subject_name]
                    try:
                        scores_dict[subject_id] = float(score.strip())
                    except:
                        pass
    except:
        pass
    return scores_dict

parse_scores_udf = udf(parse_scores_with_subject_id, MapType(IntegerType(), DoubleType()))

# Transform
df_student_scores_silver = df_scores.withColumn("studentId", concat(col("SBD"), col("Year").cast("string"))).withColumn("scores", parse_scores_udf(col("DiemThi"))).withColumn("regionId", substring(col("SBD"), 1, 2).cast("string")).select(
    col("studentId").cast("string"),
    col("regionId").cast("string"),
    col("Year").cast("int").alias("year"),
    col("scores"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("studentId").isNotNull() & col("year").isNotNull() & col("scores").isNotNull()).dropDuplicates(["studentId"])

df_student_scores_silver.writeTo("nessie.silver_tables.student_scores").using("iceberg").createOrReplace()
print(f"Đã ghi {df_student_scores_silver.count():,} dòng vào student_scores")

# Verify
print("\nXem mẫu dữ liệu (scores giờ là Map<subjectId, score>):")
spark.table("nessie.silver_tables.student_scores").show(5, truncate=False)
spark.table("nessie.silver_tables.student_scores").groupBy("year").count().orderBy("year").show()

LOAD BẢNG STUDENT_SCORES
Đọc được 993,901 dòng từ năm 2021


Đọc được 968,471 dòng từ năm 2022
Đọc được 1,025,333 dòng từ năm 2023


Đọc được 1,061,466 dòng từ năm 2024


Đọc được 1,152,914 dòng từ năm 2025

Đã load 17 môn học để mapping


25/11/05 17:52:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, Điểm thi
 Schema: SBD, DiemThi
Expected: SBD but found: ID
CSV file: s3a://bronze/structured_data/điểm%20từng%20thí%20sinh/2021/Diem_thi_2021_62_64.csv
25/11/05 17:53:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, Điểm thi
 Schema: SBD, DiemThi
Expected: SBD but found: ID
CSV file: s3a://bronze/structured_data/điểm%20từng%20thí%20sinh/2021/Diem_thi_2021_62_64.csv
25/11/05 17:54:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/05 17:54:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/05 17:54:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/05 17:54:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/05 17:54:02 WARN RowBasedKey

Đã ghi 5,190,113 dòng vào student_scores

Xem mẫu dữ liệu (scores giờ là Map<subjectId, score>):
+------------+--------+----+------------------------------------------------------------------+--------------------------+--------------------------+
|studentId   |regionId|year|scores                                                            |created_at                |updated_at                |
+------------+--------+----+------------------------------------------------------------------+--------------------------+--------------------------+
|010000022022|01      |2022|{17 -> 7.5, 2 -> 8.25, 7 -> 7.6, 10 -> 8.5, 13 -> 8.4, 14 -> 6.75}|2025-11-05 17:52:44.001257|2025-11-05 17:52:44.001257|
|010000072022|01      |2022|{17 -> 7.5, 2 -> 9.0, 7 -> 5.0, 10 -> 8.0, 13 -> 7.2, 14 -> 6.0}  |2025-11-05 17:52:44.001257|2025-11-05 17:52:44.001257|
|010000082025|01      |2025|{17 -> 4.0, 10 -> 4.0, 14 -> 6.25}                                |2025-11-05 17:52:44.001257|2025-11-05 17:52:44.001257|
|01

+----+-------+
|year|  count|
+----+-------+
|2021| 985353|
|2022| 968471|
|2023|1021909|
|2024|1061466|
|2025|1152914|
+----+-------+

